Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

In [3]:
@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

In [4]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj, γ
game_parameters = [0.81, 4.61, 0.09, 0.99, 0.37];

In [5]:
pids = workers()
aux = pids[nprocs]

3

In [6]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

 17.177490 seconds (31.35 M allocations: 707.821 MiB, 3.44% gc time)
       From worker 3:	 21.022468 seconds (31.35 M allocations: 707.802 MiB, 2.67% gc time)
18.545611 seconds (30.02 M allocations: 642.090 MiB, 1.24% gc time)
       From worker 3:	 22.007298 seconds (30.02 M allocations: 642.090 MiB, 0.81% gc time)
23.818697 seconds (30.02 M allocations: 642.090 MiB, 0.88% gc time)
       From worker 3:	 25.349288 seconds (30.02 M allocations: 642.090 MiB, 0.71% gc time)
26.104063 seconds (30.02 M allocations: 642.090 MiB, 0.70% gc time)
       From worker 3:	 27.851960 seconds (30.02 M allocations: 642.090 MiB, 0.68% gc time)
26.523736 seconds (30.02 M allocations: 642.090 MiB, 1.02% gc time)
       From worker 3:	 29.006712 seconds (30.02 M allocations: 642.090 MiB, 0.69% gc time)
27.354012 seconds (30.02 M allocations: 642.090 MiB, 0.70% gc time)
       From worker 3:	 29.395441 seconds (30.02 M allocations: 642.090 MiB, 0.65% gc time)
27.496087 seconds (30.02 M allocations: 642.0

Simulation parameters

In [7]:
#θs0, θj0, μ
simulation_parameters = [90.0, 30.0, 7.02];

### Criando funções para simular o jogo

In [8]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function prob_cd(γ, share)

    #probability of a cram down
    return γ * share
end




function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next, pie_share = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2, 3]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [9]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [10]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [11]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];
    γ = game_parameters[5];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(6);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next, pie_share = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                if(u < prob_cd(γ, pie_share)) #cram down occurs
                    
                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals
                    result[6] = one(1.0) #dummy equal to one if cram down occurs
                else
                    
                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                    
                end

            end

        elseif(policy==2.0)
            
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [12]:
#unit testing
simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

6-element Array{Float64,1}:
 0.06958505657657658
 0.2788563616215367
 3.0
 3.0
 3.0
 0.0

In [13]:
#unit testing 
simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])

6-element Array{Float64,1}:
 0.2583162930675676
 0.766751069961283
 3.0
 4.0
 4.0
 0.0

In [14]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 8)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:6] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:6] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [15]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×8 Array{Float64,3}:
[:, :, 1] =
 0.0684875  0.0662232  0.0662318  0.0709485  …  0.26      0.259995  0.267948
 0.0681659  0.0725037  0.0681899  0.0727059     0.258316  0.266621  0.261002
 0.0675657  0.0689741  0.0668685  0.0635506     0.259155  0.264174  0.261251
 0.0716543  0.0704187  0.0704165  0.0688534     0.264951  0.261508  0.269083
 0.0683964  0.0727059  0.0683964  0.0703597     0.258646  0.265091  0.265096
 0.0706227  0.0695851  0.0677037  0.070663   …  0.262356  0.25889   0.261251
 0.0682316  0.0688534  0.0672198  0.0703597     0.267139  0.267595  0.260139
 0.0639754  0.0678816  0.0671434  0.389694      0.258866  0.266459  0.264951
 0.0659766  0.0696269  0.0707231  0.066115      0.259294  0.259713  0.265669
 0.0709328  0.0688676  0.0663813  0.0689741     0.265205  0.254608  0.267595
 0.0689991  0.070177   0.066807   0.0697566  …  0.264172  0.260157  0.257161
 0.0700821  0.0707946  0.0659766  0.0678102     0.260139  0.259345  0.265124
 0.0600829  0.0719443  0.0678752  0.06

In [16]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 9)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.cram_down = Results[s, :, 6];
            DF.Ds = Results[s,:,7];
            DF.Dj = Results[s,:,8];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = size(INCOURT_REORG,1)/ size(INCOURT,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
        
            #9. percentage of cases where a cram down occurred inside court
            mm9 = mean(INCOURT.cram_down)
        
            
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [17]:
#testando se aumentar S muda a simulação


using Random

Random.seed!(2021)

#test
S = 100
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

#conclusão: aumentar o número de simulações acima de 40 tem efeitos apenas na terceira casa decimal de alguns momentos

9-element Array{Float64,1}:
 1.559042507036741
 0.9988800855668556
 2.73053054094131
 0.9509090909090909
 0.736512271563752
 0.49932144805262707
 0.7672495779670531
 0.5564624109950596
 0.1515483107872748

Calculando a derivada dos momentos em relação aos parâmetros

In [18]:
#data_moments
data_moments = [1.479, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497, 0.0938];

In [19]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [24]:
 
function derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

    
    
    
    #https://en.wikibooks.org/wiki/Introduction_to_Numerical_Methods/Numerical_Differentiation
    #backward divided difference method

   


    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters



    A2 = @spawnat aux solve_tree(data2, game_parameters)
    A1 = solve_tree(data1, game_parameters)
    A2 = fetch(A2)
    
    
    
    simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    #porcentagem de variação no parâmetro
    pct = 1/100
    
    for p in P:-1:1

        if(p>size(game_parameters,1))

            simulation_parameters = copy(estimated_simulation_parameters)
            
            delta = simulation_parameters[p-size(game_parameters,1)] * pct

            simulation_parameters[p-size(game_parameters,1)] -= delta


            if(p == size(game_parameters,1)+1 || p == size(game_parameters,1)+2) #para ajustar caso seja habilidade inicial de S ou de J
                simulation_parameters[p-size(game_parameters,1)] = round(simulation_parameters[p-size(game_parameters,1)], digits=0)
            end
            
            
            #caso a variação percentual não seja suficiente para reduzir habilidades iniciais, delta será 1
            if(simulation_parameters[p-size(game_parameters,1)] == estimated_simulation_parameters[p-size(game_parameters,1)])
                delta = Int64(1)
                
                simulation_parameters[p-size(game_parameters,1)] -= delta
            end

            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta
            
        else

            #se for um parâmetro de jogo, precisa recalcular o jogo

            game_parameters = copy(estimated_game_parameters)
            
            delta = game_parameters[p] * pct
            game_parameters[p] -= delta

            A2 = @spawnat aux solve_tree(data2, game_parameters)
            A1 = solve_tree(data1, game_parameters)
            A2 = fetch(A2)



            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta


        end


    end
    
    
    
    return D
end
    

derivative_moments (generic function with 1 method)

In [25]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

 14.870662 seconds (30.02 M allocations: 642.090 MiB, 2.01% gc time)
       From worker 3:	 19.085414 seconds (30.02 M allocations: 642.090 MiB, 2.26% gc time)
18.875618 seconds (30.02 M allocations: 642.090 MiB, 1.12% gc time)
       From worker 3:	 22.089735 seconds (30.02 M allocations: 642.090 MiB, 1.79% gc time)
23.890306 seconds (30.02 M allocations: 642.090 MiB, 2.23% gc time)
       From worker 3:	 25.523453 seconds (30.02 M allocations: 642.090 MiB, 0.78% gc time)
25.673869 seconds (30.02 M allocations: 642.090 MiB, 0.86% gc time)
       From worker 3:	 27.621371 seconds (30.02 M allocations: 642.090 MiB, 0.77% gc time)
26.568685 seconds (30.02 M allocations: 642.090 MiB, 1.18% gc time)
       From worker 3:	 28.873806 seconds (30.02 M allocations: 642.090 MiB, 0.76% gc time)
26.921543 seconds (30.02 M allocations: 642.090 MiB, 0.70% gc time)
       From worker 3:	 29.931048 seconds (30.02 M allocations: 642.090 MiB, 0.81% gc time)
28.126558 seconds (30.02 M allocations: 642.0

       From worker 3:	 23.517121 seconds (30.02 M allocations: 642.090 MiB, 0.99% gc time)
24.693465 seconds (30.02 M allocations: 642.090 MiB, 1.08% gc time)
       From worker 3:	 25.515208 seconds (30.02 M allocations: 642.090 MiB, 0.73% gc time)
25.593809 seconds (30.02 M allocations: 642.090 MiB, 0.42% gc time)
       From worker 3:	 26.819061 seconds (30.02 M allocations: 642.090 MiB, 0.73% gc time)
26.129003 seconds (30.02 M allocations: 642.090 MiB, 1.27% gc time)
       From worker 3:	 27.443512 seconds (30.02 M allocations: 642.090 MiB, 0.66% gc time)
26.612718 seconds (30.02 M allocations: 642.090 MiB, 1.11% gc time)
       From worker 3:	 27.678250 seconds (30.02 M allocations: 642.090 MiB, 0.71% gc time)
26.609373 seconds (26.96 M allocations: 595.398 MiB, 0.44% gc time)
189.471327 seconds (237.10 M allocations: 4.971 GiB, 1.07% gc time)
      From worker 3:	 25.841307 seconds (26.96 M allocations: 595.398 MiB, 0.74% gc time)
      From worker 3:	195.145455 seconds (237.10

9×8 Array{Float64,2}:
   1.96312     0.100812     4.50954   …  -0.00540644   0.141821
  -0.0796915  -0.0109901   -0.896435      0.0014122   -0.00715311
   6.79259     0.370273    14.9606       -0.0200027    0.117998
   0.392817    0.0755932   -1.17845       0.00227273  -0.01295
 -14.6434     -0.172344    -1.84528       0.016333     0.652052
  -6.51846    -0.773327   -49.7913    …  -0.00504198   0.0706557
   2.15026    -0.029506     1.95659       0.00400345  -0.00269511
   0.44641    -0.0154542   -0.721064      0.0109173    0.0625962
   0.922895    0.112142     0.988781     -0.0139896   -0.0541406

In [26]:
Derivative[2,:]

8-element Array{Float64,1}:
 -0.0796915119143316
 -0.010990079455372814
 -0.8964350767629952
  0.0649889195612132
 -0.0920804045804161
  0.000857555124002972
  0.0014122010331413248
 -0.007153106437362573

In [27]:
using DelimitedFiles

writedlm( "Derivative-cd90.csv",  Derivative, ',')

Calculando erros padrão

In [28]:

dim = 9
W = Matrix{Int}(I, dim, dim)

function std_error(D, W, S)
    
    #D is the derivative matrix
    #W is the weighting matrix
    #S is the number of simulations
    
    
    Qs = (1 + 1/S) * inv((transpose(D) * W * D))
    
    
    std_error = sqrt.(diag(Qs))
    
    return std_error
end

std_error (generic function with 1 method)

In [29]:
std_error(Derivative, W, S)

8-element Array{Float64,1}:
   0.4682424808150578
  11.881574230989312
   0.1668535056125407
   0.6157238059753881
   0.8561041370056902
 360.48769682358477
 144.7114566051136
   9.14808059849499